In [50]:
import altair as alt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


In [46]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

You can get the model from my s3 bucket.  This does take awhile to download as it is ~5GB

In [32]:
#download for the model

file_name = "UHCS_Microstructure_Model.keras"

s3 = boto3.resource('s3',config=Config(signature_version=UNSIGNED))
bucket = s3.Bucket("robertgwolf-tdi-capstone-project")
bucket.download_file(file_name,file_name)

In [48]:
col_transformer_data = pd.read_csv('col_transformer_data.csv')

In [51]:
ColScaler = ColumnTransformer([('scaler',StandardScaler(),['inverse_anneal_temperature_Kelvin',
                                                                   'log_time',
                                                                  ])])

ColScaler.set_output(transform ='pandas')

ColScaler.fit(col_transformer_data)


ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                 ['inverse_anneal_temperature_Kelvin',
                                  'log_time'])])

In [ ]:
model = tf.keras.models.load_model('Microstructre_Model.keras')

In [ ]:
DIRECTORY = 'sample_micrographs'

image_generator = DataImageGeneartor()
images = image_generator.flow_from_dataframe(dataframe = images_df,
                                                        directory = DIRECTORY,
                                                        x_col = 'path',
                                                        class_mode = 'input',
                                                        color_mode = 'rgb',
                                                        shuffle = False,
                                                        target_size = (522,645),
                                                        batch_size = 1)

In [ ]:
predictions = model.predict(images)

In [ ]:
predictions = ColScaler.named_transformers_['scaler'].inverse_transform(predictions[:,:2])
predictions_df = pd.DataFrame(predictictions, columns = ['inverse_temperature','anneal_time_minutes'])
predictions_df['time'] = predictions_df['anneal_time_minutes'].apply(lambda x: np.exp(x))
predictions_df['temperature'] = 1/predictions_df['inverse_temperature']

In [14]:
test_predictions = pd.read_csv('test_dataframe.csv')


In [15]:


offset = -150

test_predictions_chart = alt.Chart(test_predictions).mark_point(color = 'orangered').encode(
    x = alt.X('delta_time', 
              axis = alt.Axis(offset = offset, title ='Time Difference (Minutes)', titleY = 175),
              scale = alt.Scale(domain=[-3000, 3000])),
              
    y = alt.Y('delta_temperature', 
              axis = alt.Axis(offset = offset,title ='Temperature Difference (K)',titleX = -175),
              scale = alt.Scale(domain=[-100, 100])),
    tooltip = [alt.Tooltip('predicted_time',title = 'Predicted Time'),
               alt.Tooltip('predicted_temperature',title = 'Predicted Temperature'),
               alt.Tooltip('true_time',title = 'True Time'),
               alt.Tooltip('true_temperature',title = 'True Temperature')]
  
).interactive()


test_predictions_chart



alt.Chart(...)